In [181]:
# setup.sh - makes directories

In [ ]:
# download.sh - downloads images

In [200]:
# gen.py - generates narrative
import math, random
# a sequence of plot events in a coming of age story
# birth => misc. rite of passages => struggle ( + resolution ) => death
rite_of_passages = {
    'infancy': {  # life, firsts, parents
        'options':[ # events in chronological order
            'parents',
            'steps'   
        ],
        'filler': ['playing', 'parents'] # events which can recur
    },
    'childhood': {  # kindergarten, learning, friends, parents
        'options':[ 
            'friend'
        ],
        'filler': ['kindergarten', 'friend']
    },
    'adolescence': { # highschool, parties, friends, romance
        'options':[ 
            'sixteenth',
            'best_friend',
            'prom',
            'first_drink'
        ],
        'filler': ['party', 'friends', 'school']
    },
    'early_adult': { # independence, travel, graduation, early career
        'options':[ 
            'moving_out',
            'twenty_first',
            'graduation'
        ],
        'filler': ['party', 'work', 'university', 'friends']
    },
    'mid_adult': { # settling down, career, marriage, children
        'options':[ 
            'marriage',
            'children'
        ],
        'filler': ['work', 'friends']
    },
    'late_adult': { # retirement, grandchildren
        'options':[ 
            'retirement'
        ],
        'filler': ['grandchildren']
    }
}

struggles = [
    {
        'life_stage': 'late_adult',
        'name': 'sickness',
        'precondition': None
    },
    {
        'life_stage': 'late_adult',
        'name': 'spouse_death',
        'precondition': 'marriage'
    }
]

class Plot: 
    def __init__(self, length):
        self.length = length if length > 20 else 20 # 3 events from each life stage minimum
        self.events = []
        self.struggle = random.choice(struggles)
        self.init_events()
            
    def init_events(self):
        class Event:
            def __init__(self, name=None, index=None, life_stage=None, image=None, text=None):
                self.name = name
                self.index = index
                self.life_stage = life_stage
                self.image = image
                self.text = text            
        length_of_struggle_stage = 0
        life_stages = list(rite_of_passages)
        life_stage_map = []
        # partition stages of life
        for i in range(self.length):
            index = i
            event = ''
            if index == 0:
                event = 'birth'
            elif index == self.length - 1:
                event = 'death'
            else:
                index = index - 1
                mapped_index = math.floor((index / (self.length - 2)) * len(life_stages))
                life_stage = life_stages[mapped_index]
                event = life_stage
            
            if event == self.struggle['life_stage']:
                length_of_struggle_stage = length_of_struggle_stage + 1
                
            self.events.append(event)
            
        life_stage_map = self.events.copy()
        # insert struggle
        min_start_offset = 1 if self.struggle['precondition'] is not None else 0
        start_of_struggle_stage = self.events.index(self.struggle['life_stage'])
        struggle_index = start_of_struggle_stage + length_of_struggle_stage - 1
        self.events[struggle_index] = self.struggle['name']
        
        # insert precondition
        if self.struggle['precondition'] is not None:
            self.events[start_of_struggle_stage] = self.struggle['precondition']

        # replace remaining life stages with rites of passage, making sure that the preconditions are always appended first
        rites = rite_of_passages
        for i in range(len(self.events)):
            value = self.events[i]
            if (value in life_stages):
                if len(rites[value]['options']) > 0:
                    event = rites[value]['options'].pop(0)
                    if (event in self.events):
                        event = rites[value]['options'].pop(0)
                else:
                    event = random.choice(rites[value]['filler'])
                self.events[i] = event
        
        # create event objects from events
        for i in range(len(self.events)):
            name = self.events[i]
            life_stage = life_stage_map[i]
            image = './temp/{}/{}'.format(life_stage_map[i], self.events[i]) if life_stage != 'death' else None
            event = Event(
                name= name,
                index=i,
                life_stage = life_stage,
                image = image
#                 text
            )
            self.events[i] = event
            
k = Plot(20)
for z in k.events:
#     print(z.name)
    print('Index: {}\nLife Stage: {}\nName: {}\nImage: {}\nText: {}\n==========='.format(z.index, z.life_stage, z.name, z.image, z.text))

# draw 
# create a directory of pages

Index: 0
Life Stage: birth
Name: birth
Image: ./temp/birth/birth
Text: None
Index: 1
Life Stage: infancy
Name: parents
Image: ./temp/infancy/parents
Text: None
Index: 2
Life Stage: infancy
Name: steps
Image: ./temp/infancy/steps
Text: None
Index: 3
Life Stage: infancy
Name: parents
Image: ./temp/infancy/parents
Text: None
Index: 4
Life Stage: childhood
Name: friend
Image: ./temp/childhood/friend
Text: None
Index: 5
Life Stage: childhood
Name: friend
Image: ./temp/childhood/friend
Text: None
Index: 6
Life Stage: childhood
Name: friend
Image: ./temp/childhood/friend
Text: None
Index: 7
Life Stage: adolescence
Name: sixteenth
Image: ./temp/adolescence/sixteenth
Text: None
Index: 8
Life Stage: adolescence
Name: best_friend
Image: ./temp/adolescence/best_friend
Text: None
Index: 9
Life Stage: adolescence
Name: prom
Image: ./temp/adolescence/prom
Text: None
Index: 10
Life Stage: early_adult
Name: moving_out
Image: ./temp/early_adult/moving_out
Text: None
Index: 11
Life Stage: early_adult
Nam

In [319]:
event_sequences = {
            'infancy': [ 'parents', 'playing'], # life, firsts, parents
            'childhood': [ 'kindergarten' ],  # kindergarten, learning, friends, parents
            'adolescence': [ 'school', 'prom', 'first_drink', 'party' ],  # highschool, parties, friends, romance
            'early_adult': [ 'moving_out', 'university', 'twenty_first', 'graduation', 'work' ], # independence, travel, graduation, early career
            'mid_adult': [ 'marriage', 'children' ], # settling down, career, marriage, children
            'late_adult': [ 'retirement', 'grandchildren', 'spouse_death', 'sickness' ] # retirement, grandchildren
        }

import os
import random
import copy
def get_image_map():
    image_map = {}
    base_path = 'temp/downloads/'
    for life_stage in os.listdir(base_path):
        image_map[life_stage] = { 'filler': [] }

        for item in os.listdir(base_path+life_stage):
            if os.path.isdir('{}{}/{}'.format(base_path, life_stage, item)): # if there is a folder of images
                image_map[life_stage][item] = [] # create an array of paths
                for folder_image in os.listdir('{}{}/{}'.format(base_path, life_stage, item)):
                    image_map[life_stage][item].append('{}{}/{}/{}'.format(base_path, life_stage, item, folder_image))

            else:
                image_map[life_stage]['filler'].append('{}{}/{}'.format(base_path, life_stage, item))
        
    return image_map

image_map = get_image_map()

def random_pop(arr):
    return arr.pop(random.randrange(len(arr)))

class Story:
    def __init__(self, num_pages):
        self.num_pages = num_pages
        self.events = self.init_events()
    def init_events(self):
        class Event:
            def __init__(self, name=None, index=None, life_stage=None, images=[], moment=None):
                self.name = name
                self.index = index
                self.life_stage = life_stage
                self.images = images
                self.moment = moment
        
        event_map = copy.deepcopy(event_sequences)
        events = []
        life_stages = list(rite_of_passages)
        life_map = copy.deepcopy(image_map)
        for i in range(self.num_pages):
            life_stage = ''
            images = []
            moment = None
            if i == 0:
                life_stage = 'birth'
                birth_image = random.choice(life_map['birth']['filler'])
                images.append(birth_image)
            elif i == self.num_pages - 1:
                life_stage = 'death'
            else:
                i = i - 1 
                mapped_index = math.floor((i / (self.num_pages - 2)) * len(life_stages))
                life_stage = life_stages[mapped_index]
                if len(event_map[life_stage]) > 0: # if there are events left to be added
                    moment = event_map[life_stage].pop(0) # pick next life event and remove from array
                    while len(images) < random.randint(4, 5): # pick between 3 to 5 images
                        if len(life_map[life_stage][moment]) > 0: 
                            image = random_pop(life_map[life_stage][moment])
                        elif len(life_map[life_stage]['filler']) > 0: # if out of moment images, use filler
                            image = random_pop(life_map[life_stage]['filler']) 
                        else: # if out of filler images, break
                            break
                        images.append(image)
                    if len(life_map[life_stage][moment]) > 0:
                        life_map[life_stage]['filler'] = life_map[life_stage]['filler'] + life_map[life_stage][moment] # if there are pictures left in the life event, chuck them in the filler array
                else: # if generic page ( no event )
                    while len(images) < random.randint(2, 4): # pick between 2 to 3 images
                        if len(life_map[life_stage]['filler']) > 0: # pick first from filler images
                            image = random_pop(life_map[life_stage]['filler'])
                        else: # if out of images to pick from, break
                            print(life_stage, 'out of filler')
                            break
                        images.append(image)           

                i = i + 1

            event = Event(index=i, life_stage=life_stage, images=images, moment=moment)
            events.append(event)
        return events

z = Story(28)
for event in z.events:
    print('#{} || {} || {}'.format(event.index, event.life_stage, event.moment))
#     print('#{:02d }|| {} || {} || {}\n\n==========================\n'.format(event.index, event.life_stage, event.moment, event.images))

#0 || birth || None
#1 || infancy || parents
#2 || infancy || playing
#3 || infancy || None
#4 || infancy || None
#5 || infancy || None
#6 || childhood || kindergarten
#7 || childhood || None
#8 || childhood || None
#9 || childhood || None
#10 || adolescence || school
#11 || adolescence || prom
#12 || adolescence || first_drink
#13 || adolescence || party
#14 || early_adult || moving_out
#15 || early_adult || university
#16 || early_adult || twenty_first
#17 || early_adult || graduation
#18 || early_adult || work
#19 || mid_adult || marriage
#20 || mid_adult || children
#21 || mid_adult || None
#22 || mid_adult || None
#23 || late_adult || retirement
#24 || late_adult || grandchildren
#25 || late_adult || spouse_death
#26 || late_adult || sickness
#27 || death || None


In [429]:
# draw pages into a new directory
from PIL import Image, ImageDraw
from IPython.display import Image as PyImage
import time
import datetime
# create directory ====
timestamp = datetime.datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d-%H%M%S%f')
image_dir = 'temp/outputs/images/{}/'.format(timestamp)
if not os.path.exists(image_dir):
    os.makedirs(image_dir)

# create pages for each event in the story object ====
height = 400
width = 400
for event in k.events:
    img = Image.new('RGB', (width, height), "black")
    ctx = ImageDraw.Draw(img)
    ctx.text((width/2, height/2), str(event.index),fill=(255,255,255))
    img.save('{}{:02d}.png'.format(image_dir, event.index), 'PNG')


# for 






# # Create image doc
# 

# ctx = ImageDraw.Draw(img)
# ctx.text((width/2, height/2), str(1),fill=(255,255,255))
# img.save('one.png', 'PNG')

# PyImage(filename='one.png')



In [430]:
# https://stackoverflow.com/questions/27327513/create-pdf-from-a-list-of-images
from fpdf import FPDF
from PIL import Image
def makePdf(pdfFileName, listPages, dir = '', pdfpath=''):
    cover = Image.open(dir + str(listPages[0]))
    width, height = cover.size

    pdf = FPDF(unit = "pt", format = [width, height])

    for page in listPages:
        pdf.add_page()
        pdf.image(dir + str(page), 0, 0)
    print(pdfpath + ".pdf")
    pdf.output(pdfpath + ".pdf", "F")

image_paths = os.listdir(image_dir)
pdf_output_path = 'temp/outputs/pdfs/{}'.format(timestamp)
makePdf(timestamp, image_paths, dir=image_dir, pdfpath=pdf_output_path)



temp/outputs/pdfs/2017-10-30-210804650544.pdf
